In [1]:
MARKET = 'SNP'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = DATAPATH.joinpath('temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, PAPER, CID = ibp.HOST, ibp.PORT, ibp.PAPER, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_fresh.pkl', 'df_ohlcs.pkl',
       'df_opt_prices.pkl', 'df_symlots.pkl', 'df_und_margins.pkl',
       'df_unds.pkl', 'dfrq.pkl', 'qopts.pkl', 'qopts1.pkl',
       'z_temp_df_opt_prices.pkl'], dtype='<U24')

# Making df_prices_opt
* [ ] Set up contract data
* [ ] Run execAsync

## execAsync and its support files

In [10]:
# ** EXECUTION
# * Core engine that processes functions and delivers results

# .preprocessing data for the core engine
def pre_process(cts):
    """Generates tuples for input to the engine"""

    try:
        symbol = cts.symbol
        output = ((cts, None),)

    except AttributeError as ae1:  # it's an iterable!
        try:
            symbols = [c.symbol for c in cts]

            if len(symbols) == 1:
                output = ((cts[0], None),)
            else:
                output = ((c, None) for c in cts)

        except AttributeError as ae2:  # 2nd value is MarketOrder!
            try:
                output = tuple(cts)
            except:
                print(f"Unknown error in {ae2}")
                output = None

    return tuple(output)


# .make name for symbol being processed by the engine
def make_name(cts):
    """Generates name for contract(s)"""
    try:
        output = [
            c.symbol
            + c.lastTradeDateOrContractMonth[-4:]
            + c.right
            + str(c.strike)
            + ".."
            for c in cts
        ]

    except TypeError as te:  # single non-iterable element
        if cts:  # not empty!
            output = (
                cts.symbol
                + cts.lastTradeDateOrContractMonth[-4:]
                + cts.right
                + str(cts.strike)
            )
        else:
            output = cts

    except AttributeError as ae1:  # multiple (p, s) combination
        try:
            output = [
                c[0].symbol
                + c[0].lastTradeDateOrContractMonth[-4:]
                + c[0].right
                + str(c[0].strike)
                + ".."
                for c in cts
            ]
        except TypeError as ae2:
            output = (
                cts[0].symbol
                + cts[0].lastTradeDateOrContractMonth[-4:]
                + cts[0].right
                + str(cts[0].strike)
            )

    return output


# .the core engine
async def executeAsync(
    ib: IB(),
    algo: Callable[..., Coroutine],  # coro name
    cts: Union[Contract, pd.Series, list, tuple],  # list of contracts
    CONCURRENT: int = 44,  # to prevent overflows put 44 * (TIMEOUT-1)
    TIMEOUT: None = None,  # if None, no progress messages shown
    post_process: Callable[
        [set, pathlib.Path, str], pd.DataFrame
    ] = None,  # If checkpoint is needed
    DATAPATH: pathlib.Path = None,  # Necessary for post_process
    OP_FILENAME: str = "",  # output file name
    SHOW_TQDM: bool = True,  # Show tqdm bar instead of individual messages
    REUSE: bool = False, # Reuse the OP_FILENAME supplied
    **kwargs) -> pd.DataFrame:

    tasks = set()
    results = set()
    remaining = pre_process(cts)
    last_len_tasks = 0  # tracking last length for error catch
    
    # Set pbar
    if SHOW_TQDM:
        pbar = tqdm(
            total=len(remaining), desc=f"{algo.__name__}: ", 
            bar_format=BAR_FORMAT,
            leave=True
        )

    # Get the results
    while len(remaining):

        # Tasks limited by concurrency
        if len(remaining) <= CONCURRENT:

            tasks.update(
                asyncio.create_task(algo(ib, c, **kwargs), name=make_name(c))
                for c in remaining
            )

        else:

            tasks.update(
                asyncio.create_task(algo(ib, c, **kwargs), name=make_name(c))
                for c in remaining[:CONCURRENT]
            )

        # Execute tasks
        while len(tasks):

            done, tasks = await asyncio.wait(
                tasks, timeout=TIMEOUT, return_when=asyncio.ALL_COMPLETED
            )

            # Remove dones from remaining
            done_names = [d.get_name() for d in done]
            remaining = [c for c in remaining if make_name(
                c) not in done_names]

            # Update results and checkpoint
            results.update(done)

            # Checkpoint the results
            if post_process:
                
                output = post_process(results=results, DATAPATH=DATAPATH, 
                                      REUSE = REUSE, LAST_RUN = False, 
                                      OP_FILENAME = OP_FILENAME)
                
                REUSE = False # for second run onwards
            else:
                output = results

            if TIMEOUT:

                if remaining:
                    
                    if SHOW_TQDM:
                        pbar.update(len(done))
                        
                    else:
                        print(
                            f"\nDone {algo.__name__} for {done_names[:2]} {len(results)} out of {len(cts)}. Pending {[make_name(c) for c in remaining][:2]}"
                        )

                # something wrong. Task is not progressing
                if (len(tasks) == last_len_tasks) & (len(tasks) > 0):
                    print(
                        f"\n @ ALERT @: Tasks are not progressing. Pending tasks will be killed in 5 seconds !\n"
                    )
                    dn, pend = await asyncio.wait(tasks, timeout=5.0)
                    if len(dn) > 0:
                        results.update(dn)

                    tasks.difference_update(dn)
                    tasks.difference_update(pend)

                    pend_names = [p.get_name() for p in pend]
                    # remove pending from remaining
                    remaining = [c for c in remaining if make_name(
                        c) not in pend_names]

                # re-initialize last length of tasks
                last_len_tasks = len(tasks)
                
    
    # Make the final output, based on REUSE status
    
    if OP_FILENAME:
        df = post_process(results=set(), # Empty dataset
                              DATAPATH=DATAPATH, 
                              REUSE = REUSE, LAST_RUN = True, 
                              OP_FILENAME = OP_FILENAME)
    else:
        df = output

    if SHOW_TQDM:

        pbar.update(len(done))
        pbar.refresh()
        pbar.close()

    return df


# .Process output into dataframes
def save_df(results: set, DATAPATH: pathlib.Path, REUSE: bool,  
            LAST_RUN: bool, OP_FILENAME: str = "") -> pd.DataFrame():

    if results:
        df = pd.concat([r.result() for r in results if r], ignore_index=True)
        
        if OP_FILENAME:
            
            if REUSE:
                
                # load the existing file
                df_old = pd.read_pickle(DATAPATH.joinpath(OP_FILENAME))
                
                # Save old temporarily
                df_old.to_pickle(DATAPATH.joinpath("z_temp_"+ OP_FILENAME))
            
            df.to_pickle(DATAPATH.joinpath(OP_FILENAME))
                
    else:
        
        if LAST_RUN: # Merge new and old df (if available)
            
            if OP_FILENAME:
            
                try:
                    df_old = pd.read_pickle(DATAPATH.joinpath("z_temp_"+ OP_FILENAME))
                    os.remove(DATAPATH.joinpath("z_temp_"+ OP_FILENAME)) # cleanup temp file

                except FileNotFoundError:
                    df_old = pd.DataFrame([])

                df_new = pd.read_pickle(DATAPATH.joinpath(OP_FILENAME))
                df = df_new.append(df_old).reset_index(drop=True)
                df.to_pickle(DATAPATH.joinpath(OP_FILENAME))
            
        else:
            df = pd.DataFrame([])  # results are not yet ready!
        
    return df

In [36]:
# .price coroutine
async def price(ib: IB, co, **kwargs) -> pd.DataFrame:
    """Optimal execAsync: CONCURRENT=40 and TIMEOUT=8 gives ~250 contract prices/min"""

    TEMPL_PATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "template",
                                             "df_price.pkl")
    df_empty = pd.read_pickle(TEMPL_PATH)
    cols = list(df_empty)

    try:
        FILL_DELAY = kwargs["FILL_DELAY"]
    except KeyError as ke:
        print(
            f"\nWarning: No FILL_DELAY supplied! 5.5 second default is taken\n"
        )
        FILL_DELAY = 5.5

    try:

        if isinstance(co, tuple):
            c = co[0]
        else:
            c = co

        df = (util.df([c]).iloc[:, :6].rename(
            columns={"lastTradeDateOrContractMonth": "expiry"}))

    except (TypeError, AttributeError, ValueError) as err:
        print(f"\nError: contract {co} supplied is incorrect!" + f"\n{err}" +
              f"\n... and empty df will be returned !!!")

        df = df_empty

        return df  # ! Aborted return with empty df for contract error

    tick = ib.reqMktData(c, genericTickList="106")

    await asyncio.sleep(FILL_DELAY)

    df = df.assign(localSymbol=c.localSymbol, contract=c)

    try:
        dfpr = util.df([tick])

        if dfpr.modelGreeks[0] is None:
            iv = dfpr.impliedVolatility
        else:
            iv = dfpr.modelGreeks[0].impliedVol

        df = df.assign(
            time=dfpr.time,
            greeks=dfpr.modelGreeks,
            bid=dfpr.bid,
            ask=dfpr.ask,
            close=dfpr["close"],
            last=dfpr["last"],
            price=dfpr["last"].combine_first(dfpr["close"]),
            iv=iv,
        )

    except AttributeError as e:

        print(
            f"\nError in {c.localSymbol}: {e}. df will have no price and iv!\n"
        )

        df = df.assign(
            time=np.nan,
            greeks=np.nan,
            bid=np.nan,
            ask=np.nan,
            close=np.nan,
            last=np.nan,
            price=np.nan,
            iv=np.nan,
        )

    ib.cancelMktData(c)

    return df


In [40]:
# imports
from tqdm.notebook import tqdm

BAR_FORMAT = "{desc:<10}{percentage:3.0f}%|{bar:30}{r_bar}{bar:-10b}"

In [14]:
# inputs
MARKET = 'SNP'
RUN_ON_PAPER = True
REUSE = True
OP_FILENAME = 'df_opt_prices.pkl'

In [15]:
# * SETUP

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml

HOST, CID = ibp.HOST, ibp.CID

if RUN_ON_PAPER:
    print(f"\nMargin and Price for {MARKET} options using Paper account\n")
    PORT = ibp.PAPER
else:
    PORT = ibp.PORT

LOGPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "log")
DATAPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", MARKET.lower())

# * SETUP LOGS AND CLEAR THEM
LOGFILE = LOGPATH.joinpath(MARKET.lower() + "_temp.log")
util.logToFile(path=LOGFILE, level=30)
with open(LOGFILE, "w"):
    pass


Margin and Price for SNP options using Paper account



# Building contracts and executing with REUSE

In [52]:
# * BUILD UPON EXISTING CONTRACTS
COLS = ['symbol', 'strike', 'right', 'expiry']


# .load existing prices
if REUSE:
    
    try:
        df_opt1 = pd.read_pickle(DATAPATH.joinpath(OP_FILENAME))
    except FileNotFoundError:
        df_opt1 = pd.DataFrame([])
    
    try:
        df_opt2 = pd.read_pickle(DATAPATH.joinpath("z_temp_"+ OP_FILENAME))
    except FileNotFoundError:
        df_opt2 = pd.DataFrame([])
        
    df_opt3 = df_opt1.append(df_opt2)
    

# .cleanup duplicates
df_opt3 = df_opt3.drop_duplicates(COLS).reset_index(drop=True)

# . load ALL option contracts
qopts = pd.read_pickle(DATAPATH.joinpath('qopts.pkl'))

# ... convert it to df
df_q_opts = util.df(qopts.to_list())\
                .assign(contract=qopts)\
                .rename(columns={"lastTradeDateOrContractMonth": "expiry"})

# ... remove existing price df from df_q_opts

m = ~df_q_opts[COLS].apply(tuple,1).isin(df_opt3[COLS].apply(tuple, 1))
df_opt4 = df_q_opts[m]

# ... overwrite temp opt price df with existing prices
df_opt5 = df_q_opts[~m].drop_duplicates(COLS).reset_index(drop=True)
df_opt5.to_pickle(DATAPATH.joinpath("z_temp_"+ OP_FILENAME))

price_contracts = df_opt4.contract

# * GET THE PRICE AND IV
with IB().connect(HOST, PORT, CID) as ib:
    df_opt_prices = ib.run(
        executeAsync(
            ib=ib,
            algo=price,
            cts=price_contracts,
            post_process=save_df,
            CONCURRENT=40 * 4,
            TIMEOUT=11,
            DATAPATH=DATAPATH,
            REUSE=True,
            OP_FILENAME="df_opt_prices.pkl",
            **{"FILL_DELAY": 11},
        ))

KeyboardInterrupt: 

## One contract price

In [ ]:
# imports
import os
from support import get_dte
from tqdm.notebook import tqdm
BAR_FORMAT = "{l_bar}{bar:-20}{r_bar}"

In [ ]:
# build a set of contracts to test
df = util.df(qopts.to_list()).iloc[:, 1:6].rename(columns={'lastTradeDateOrContractMonth': 'expiry'})

df['contract'] = qopts
df['dte'] = df.expiry.apply(get_dte)
df = df[df.dte>5].reset_index(drop=True)

In [ ]:
# set the inputs
ct = df.sample(1).contract.iloc[0]
kwargs = {'FILL_DELAY': 7}

In [ ]:
# .Price
async def price(ib: IB, co, **kwargs) -> pd.DataFrame:
    """Optimal execAsync: CONCURRENT=40 and TIMEOUT=8 gives ~250 contract prices/min"""

    TEMPL_PATH = pathlib.Path.cwd().joinpath(
        THIS_FOLDER, "data", "template", "df_price.pkl"
    )
    df_empty = pd.read_pickle(TEMPL_PATH)
    cols = list(df_empty)

    try:
        FILL_DELAY = kwargs["FILL_DELAY"]
    except KeyError as ke:
        print(f"\nWarning: No FILL_DELAY supplied! 5.5 second default is taken\n")
        FILL_DELAY = 5.5

    try:

        if isinstance(co, tuple):
            c = co[0]
        else:
            c = co

        df = (
            util.df([c])
            .iloc[:, :6]
            .rename(columns={"lastTradeDateOrContractMonth": "expiry"})
        )

    except (TypeError, AttributeError, ValueError) as err:
        print(
            f"\nError: contract {co} supplied is incorrect!"
            + f"\n{err}"
            + f"\n... and empty df will be returned !!!"
        )

        df = df_empty

        return df  # ! Aborted return with empty df for contract error

    tick = ib.reqMktData(c, genericTickList="106")

    await asyncio.sleep(FILL_DELAY)

    df = df.assign(localSymbol=c.localSymbol, contract=c)

    try:
        dfpr = util.df([tick])

        if dfpr.modelGreeks[0] is None:
            iv = dfpr.impliedVolatility
        else:
            iv = dfpr.modelGreeks[0].impliedVol

        df = df.assign(
            time=dfpr.time,
            greeks=dfpr.modelGreeks,
            bid=dfpr.bid,
            ask=dfpr.ask,
            close=dfpr["close"],
            last=dfpr["last"],
            price=dfpr["last"].combine_first(dfpr["close"]),
            iv=iv,
        )

    except AttributeError as e:

        print(
            f"\nError in {c.localSymbol}: {e}. df will have no price and iv!\n")

        df = df.assign(
            time=np.nan,
            greeks=np.nan,
            bid=np.nan,
            ask=np.nan,
            close=np.nan,
            last=np.nan,
            price=np.nan,
            iv=np.nan,
        )

    ib.cancelMktData(c)

    return df

In [ ]:
ct

In [ ]:
%%time
with IB().connect(HOST, PORT, CID) as ib:
    dfp = ib.run(price(ib, ct, **kwargs))
dfp